Carpages scraping project
carpages.ca is a popular canadian website for buying used and new vehicles in canada. In this project, I'll be scraping the data using 
python libraries such as BeautifulSoup, requests and pandas   

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [7]:
# Lets check to see if we can get a response from the website
website = "https://www.carpages.ca/used-cars/search/?num_results=50&category_id=5&p=1"
response = requests.get(website)

In [9]:
response.status_code

200

In [8]:
soup = BeautifulSoup(response.content,'html.parser')

In [ ]:
soup

In [95]:
cars = soup.find_all('div', class_='media soft push-none rule')
car = cars[1]
title = car.find("a").get("title")
#car.find("a")
location = car.find("p").text
dealer = car.find_all("h5")[1].text
link = "https://www.carpages.ca"+car.find("a").get("href")
price = int(car.find("strong").text.strip().replace("$","").replace(",",""))
city,province = location.split(" ")
province
city = city[0:-1]
city
price

'20900'

In [99]:
mileage_string = car.find("div",class_="grey l-column l-column--small-6 l-column--medium-4").find_all("span")
mileage_num = ""
for m in mileage_string:
    if m != "K" or m != "M":
        mileage_num += m.text

print(mileage_num)
mileage = int(mileage_num[0:-2].replace(",",""))
mileage

80,200KM


80200

In [106]:
def getMileage(car):
    # Takes a car and gives us the mileage from the tag
    unFilteredMileageTags = car.find("div",class_="grey l-column l-column--small-6 l-column--medium-4").find_all("span")
    mileage = ""
    for tag in unFilteredMileageTags:
        mileage += tag.text
    #mileage = int(mileage[0:2].replace(",",""))
    return mileage

def getLocationField(car):
    location = car.find("p").text
    city,province = location.split(" ")
    return city[0:-1],province

def parse_page(no_listings,body_type,page_num):
    # page_num is the page number
    # result_num is the number of the 
    website = "https://www.carpages.ca/used-cars/search/?num_results="+str(no_listings)+"&category_id="+str(body_type)+"&p="+str(page_num)
    response = requests.get(website)
    status = response.status_code

    titles,prices,links,mileages,dealers,locations = [],[],[],[],[],[]
    if status == 200:
        # this means we have the ability to scrape the website
        cars = soup.find_all('div', class_='media soft push-none rule')
        for car in cars:
            # We need to get the following data, title,price,link,mileage,dealer,location(city), location(province)
            title = car.find("a").get("title") # from the title (we can see if we can filter the model and the make from the )
            price = int(car.find("strong").text.strip().replace("$","").replace(",",""))
            link = "https://www.carpages.ca"+car.find("a").get("href")
            mileage = getMileage(car) # We make a function for the mileage as it takes some work
            dealer = car.find_all("h5")[1].text
            #city,province = getLocationField(car) # We make a similar function to parse the city and the province
            location = car.find("p").text

            titles.append(title)
            prices.append(price)
            links.append(link)
            mileages.append(mileage)
            dealers.append(dealer)
            locations.append(location)

    data_dict = {"titles":titles,"prices":prices,"links":links,"mileages":mileages,"dealers":dealers,"locations":location}
    data = pd.DataFrame(data_dict)
    return data

# sedans are 5, 50 cars, 1 result
data = parse_page("10","5","1")
data

,titles,prices,links,mileages,dealers,locations
0,Used 2016 Mercedes-Benz C-Class C450 AMG - PAN...,36895,https://www.carpages.ca/used-cars/ontario/nort...,"98,000KM",Monaco Motorcars Inc,"Hamilton, ON"
1,Used 2020 Hyundai Elantra Preferred w/Sun & Sa...,20900,https://www.carpages.ca/used-cars/ontario/bram...,"80,200KM",#9 Auto Sales,"Hamilton, ON"
2,Used 2021 Toyota Corolla LE CVT SUNROOF for sa...,25900,https://www.carpages.ca/used-cars/ontario/bram...,"84,200KM",#9 Auto Sales,"Hamilton, ON"
3,Used 2011 Suzuki Kizashi Sport LOS KMs | ONLY ...,12490,https://www.carpages.ca/used-cars/ontario/wate...,"93,000KM",Carimex,"Hamilton, ON"
4,New 2023 Dodge Charger Scat Pack 392 for sale ...,79900,https://www.carpages.ca/new-cars/british-colum...,,Langley Chrysler,"Hamilton, ON"
5,Used 2013 Honda Civic Sedan EX - Sunroof - H...,15079,https://www.carpages.ca/used-cars/ontario/nort...,"85,575KM",Subaru of North Bay,"Hamilton, ON"
6,Used 2018 Honda Civic | leather | sunroof | na...,27999,https://www.carpages.ca/used-cars/ontario/st-t...,"75,012KM",519 Cars,"Hamilton, ON"
7,Used 2021 Kia Forte EX No Accident Sunroof Ca...,24995,https://www.carpages.ca/used-cars/ontario/miss...,"71,904KM",Tabangi Motors,"Hamilton, ON"
8,Used 2021 Hyundai Elantra Preferred No Accid...,24995,https://www.carpages.ca/used-cars/ontario/miss...,"81,696KM",Tabangi Motors,"Hamilton, ON"
9,Used 2017 Volkswagen Passat HIGHLINE|NAVI|BSM|...,22888,https://www.carpages.ca/used-cars/ontario/oakv...,"110,126KM",S.S. Auto Group,"Hamilton, ON"


In [100]:
dict = {"a":[1,2,3],"b":[1,2,3]}
df = pd.DataFrame(dict)
df

,a,b
0,1,1
1,2,2
2,3,3
